In [16]:
# !pip install torch torchvision matplotlib

In [17]:
import os
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

data_dir = "newDataset"
num_classes = 4
batch_size = 16
epochs = 6
device = torch.device('cpu')

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
class_names = dataset.classes  

# Model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Acc: {epoch_acc:.4f}")

torch.save(model.state_dict(), "newfiber_model.pth")

Epoch 1/6 - Loss: 1.4976 - Acc: 0.0000
Epoch 2/6 - Loss: 0.2445 - Acc: 1.0000
Epoch 3/6 - Loss: 0.0722 - Acc: 1.0000
Epoch 4/6 - Loss: 0.0333 - Acc: 1.0000
Epoch 5/6 - Loss: 0.0193 - Acc: 1.0000
Epoch 6/6 - Loss: 0.0128 - Acc: 1.0000


In [18]:
import os
from PIL import Image
import torch
from torchvision import transforms, models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 4
model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("newfiber_model.pth", map_location=device))
model.to(device)
model.eval()

class_names = ['angle_plied', 'cross_plied', 'longitudinal', 'transverse']

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.convert("RGB")),
    transforms.ToTensor(),
])

In [19]:

def predict_folder(folder_path):
    results = {}
    for fname in os.listdir(folder_path):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        img_path = os.path.join(folder_path, fname)
        img = Image.open(img_path)
        img = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(img)
            _, pred = torch.max(output, 1)
        results[fname] = class_names[pred.item()]
    return results

# Example usage
data_dir = "newDataset"  # 
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder}")
        predictions = predict_folder(folder_path)
        for img_name, pred_class in predictions.items():
            print(f"{img_name}: {pred_class}")


Processing folder: angled_plied
angled.png: angle_plied
image.png: angle_plied
Processing folder: cross_plied
cross.png: cross_plied
image.png: cross_plied
image1.png: cross_plied
Processing folder: longitudinal
long.png: longitudinal
Processing folder: transverse
trans.png: transverse


In [23]:
from PIL import Image
import torch

def predict_custom_image(image_path, model, transform, class_names, device):
    model.eval()
    img = Image.open(image_path)
    img = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(img)
        _, pred = torch.max(output, 1)
    return class_names[pred.item()]
    
print(f"Prediction: {predict_custom_image('angled.png', model, transform, class_names, device)} fibre orientation")
print(f"Prediction: {predict_custom_image('long.png', model, transform, class_names, device)} fibre orientation")
print(f"Prediction: {predict_custom_image('trans.png', model, transform, class_names, device)} fibre orientation")
print(f"Prediction: {predict_custom_image('cross.png', model, transform, class_names, device)} fibre orientation")

Prediction: angle_plied fibre orientation
Prediction: longitudinal fibre orientation
Prediction: transverse fibre orientation
Prediction: cross_plied fibre orientation
